### Caso de Estudios:
* Tenemos el set de datos de un banco, el banco ha notado que una gran cantidad de clientes están dejando el banco y quieren ver porque.
* Nuestro trabajo es crear un modelo que prediga que clientes están en mayor riesgo de dejar el banco, el mismo modelo puede ser aplicado para evaluar si una persona debería tener un préstamo aprobado.
* Estamos intentando predecir el resultado de nuestra variable dependiente "Exited" baso en la info de las variables independientes.



### Vamos a instalar las librerias:
* <b>Theano</b> (computación numérica) corre también en el GPU.
* <b>TensorFlow</b> (computación numérica)<br><br>

- Ambas librerias son utilizadas para investigación y desarrollo, si quieres crear tus propias redes neuronales desde cero podrías hacerlo con estas 2 librerias, pero nosotros vamos a utilizar unas que ya están pre hechas.<br><br>

- Vamos a utilizar la libreria llamada Keras,

* <b>Keras</b> (basado en theano y TensorFlow, modelos de Deep Learning).

#### Para instalar Theano:
> pip install theano

#### Para instalar Keras:
> pip install keras<br>
> o<br>
> conda install keras

#### Para instalar TensorFlow:
> conda install tensorflow <br>
> Y <br>
> conda install tensorflow-estimator=2.1.0

In [2]:
# Importando las librerias en Python
import numpy as np
import pandas as pd



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 1. Dando formato a los datos de Churn_Modelling.csv utilizando redes neuronales

In [11]:
# Importación de datos de la base de datos:
dataset = pd.read_csv('/content/drive/MyDrive/Churn_Modelling.csv')
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [12]:
dataset.size

140000

In [13]:
# Seleccionamos las variables independientes (columnas) que nos vayan a servir
X = dataset.iloc[:, 3:13].values
print(X[1])

[608 'Spain' 'Female' 41 1 83807.86 1 0 1 112542.58]


In [14]:
# SEleccionar la variable dependiente
y = dataset.iloc[:, 13].values
y

array([1, 0, 1, ..., 1, 1, 0])

In [15]:
# Codificación de datos categóricos
# Codificación de las variables independientes
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
# Categorizamos los paises, codificados por números
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
X[1]


array([608, 2, 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58],
      dtype=object)

In [16]:
# Categorizamos los generos, codificados por números
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
X[1]

array([608, 2, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58], dtype=object)

In [17]:
# OneHotEncoder codifica características enteras categóricas como una matriz numérica one-hot
#onehotencoder = OneHotEncoder(categorical_features = [1])
#X = onehotencoder.fit_transform(X).toarray()
 
onehotencoder = ColumnTransformer([("Country", OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(onehotencoder.fit_transform(X))
X[1]

array([0.0, 0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
      dtype=object)

In [18]:
# Eliminar la variable ficticia que hemos creado
X = X[:,1:]
X[1]

array([0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
      dtype=object)

In [ ]:
# Dividir el conjunto de datos en el conjunto de entrenamiento y pruebas
from sklearn.model_selection import train_test_split
# Dedicar 20% datos a la prueba y el restante al entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# Escalado de Categorias
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



In [ ]:
X_test

array([[ 1.75486502, -0.57369368, -0.55204276, ...,  0.64259497,
         0.9687384 ,  1.61085707],
       [-0.5698444 , -0.57369368, -1.31490297, ...,  0.64259497,
        -1.03227043,  0.49587037],
       [-0.5698444 ,  1.74309049,  0.57162971, ...,  0.64259497,
         0.9687384 , -0.42478674],
       ...,
       [-0.5698444 ,  1.74309049, -0.74791227, ...,  0.64259497,
        -1.03227043,  0.71888467],
       [ 1.75486502, -0.57369368, -0.00566991, ...,  0.64259497,
         0.9687384 , -1.54507805],
       [ 1.75486502, -0.57369368, -0.79945688, ...,  0.64259497,
        -1.03227043,  1.61255917]])

### 2. Creando Red Neuronal Artificial

In [ ]:
# importando librerias y paquetes de Keras
# Sequential ayuda a iniciarlizar la red neuronal
# Dense ayuda a agregar capas ocultas en nuestra red
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Inicializar la Red Neuronal, definiendo como una secuencia de capas
clasificador = Sequential()

### 2.1. Pasos de Retropropagación (EPOCHs)

El algoritmo de Retropropagación que permite actualizar los pesos en la variable W de la sinapsis.

* __Paso 1.__ Inicia los pesos con números aleatorios cercanos a 0.
* __Paso 2.__ Colocamos cada variable independiente de tu set de datos en la capa de input.
* __Paso 3.__ Propagación hacia adelante, pasa la información individual (sinapsis) realizando la suma de los pesos con las variables, después pasa por la función de activación, y por último pasa la ^y (resultado de la predicción). Las neuronas son activadas de manera que su impacto es limitado por los pesos.
* __Paso 4.__ Comparamos la ^y con y, o sea, los datos (resultados) predecidos con los datos (resultados) reales que tenemos. Y utilizamos la función C que es el margen de error (Siempre disminuir).
* __Paso 5.__ La retropropagación, actualiza los pesos de acuerdo a C (margen de error), o sea, cuan responsables son por el margen de erro.
> El rango de aprendizaje decide por cuanto actualizamos los pesos.
* __Paso 6.__ Tienes dos opciones para minizar la función C que es el margen del error:
> repite pasos 1 a 5 y actualizamos los pesos después de cada observación (aprendizaje reforzado o ***reinforcement learning***).<br>
> o<br>
> Repite los pasos de 1 a 5 solo después de un lote de observaciones (aprendizaje por lote o ***descenso de gradiente estocástico***).<br>
* __Paso 7.__ Cuando todo el set de datos pasa por la red neuronal, eso hace 1 EPOCH. Repite más epochs hasta refinal el modelo.

In [ ]:
# Agregando Capa input y Primero Capa Oculta
# units son la cantidad de nodos que se desean agregar (e.g. puede ser la suma de los nodos de ingreso y salida).
# kernel_initializer es el Paso 1, de tipo distribución uniforme
# activation o Función de activación de tipo rectificante = relu
# input_dim crea la capa de ingreso, en este caso 11 columnas
clasificador.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))

In [ ]:
clasificador

In [ ]:
# Agregando segunda capa oculta
clasificador.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

In [ ]:
# Agregando s3 capa oculta
clasificador.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

In [ ]:
# Agregando Capa de Salida con Función de Activación Sigmoide
clasificador.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

In [ ]:
# Aplicamos el Descenso de Gradiente Estocástica, significa Compilar Red Neuronal.
# optimazer permite definir el algorimo de descenso de gradiente.
# loss es el margen de error "C". En redes convulucionales entendemos como "Binary CrossEntropy" o Entropia Cruzada.
# binary_crossentropy para una salida (Verdadero o Falso), para varios resultados usamos categorical_crossentropy
# metric es el criterio para evaluar el modelo, 
clasificador.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Ajustamos Red Neuronal en el Set de Entrenamiento
# batch_size es el número de ejemplos por gradiente actualizable.
clasificador.fit(X_train, y_train, batch_size=10, epochs=300)

Epoch 1/300
800/800 [==============================] - 1s 2ms/step - loss: 0.3690 - accuracy: 0.8504
Epoch 2/300
800/800 [==============================] - 1s 2ms/step - loss: 0.3626 - accuracy: 0.8566
Epoch 3/300
800/800 [==============================] - 1s 2ms/step - loss: 0.3601 - accuracy: 0.8546
Epoch 4/300
800/800 [==============================] - 1s 2ms/step - loss: 0.3585 - accuracy: 0.8580
Epoch 5/300
800/800 [==============================] - 1s 2ms/step - loss: 0.3545 - accuracy: 0.8571
Epoch 6/300
800/800 [==============================] - 1s 2ms/step - loss: 0.3523 - accuracy: 0.8599
Epoch 7/300
800/800 [==============================] - 1s 2ms/step - loss: 0.3510 - accuracy: 0.8574
Epoch 8/300
800/800 [==============================] - 1s 2ms/step - loss: 0.3502 - accuracy: 0.8609
Epoch 9/300
800/800 [==============================] - 1s 2ms/step - loss: 0.3496 - accuracy: 0.8605
Epoch 10/300
800/800 [==============================] - 1s 2ms/step - loss: 0.3486 - accura

### 3. Haciendo Predicciones y Evolucionando el Modelo

* Entrenado el modelo en el Set de Entrenamiento, ahora vamos a predecir el ser de pruebas.

In [ ]:
# Prediciendo Set de Pruebas
y_pred = clasificador.predict(X_test)
y_pred

array([[0.317367  ],
       [0.4186476 ],
       [0.22438797],
       ...,
       [0.23305228],
       [0.27938122],
       [0.19396108]], dtype=float32)

In [ ]:
y_pred = (y_pred>0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [ ]:
# Matri de Confusión: Resumen de los resultados de la predicción del problema de clasificación del modelo.
from sklearn.metrics import confusion_matrix
mc = confusion_matrix(y_test, y_pred)
mc
# ¿que quieren decir estos resultado?

array([[1475,  120],
       [ 167,  238]])

In [ ]:
(1475 + 238)/2000



0.8565

### 4. Aplicando nuestro modelo de Predicción

Ingresa un cliente al banco con los siguiente valores:
* Geografía: Francia
* Puntaje de Crédito: 600
* Género: Masculino
* Edad: 40 años
* Tenencia: 3 años
* Saldo: \$60000
* Número de productos: 2
* ¿Tiene este cliente una tarjeta de crédito? Sí
* ¿Este cliente es un miembro activo? Sí
* Salario estimado: $50000

In [ ]:
nuevo_cliente = clasificador.predict(sc.transform(np.array([[0,0,600,1,40,3,60000,2,1,1,50000]])))
nuevo_cliente = (nuevo_cliente>0.5)
nuevo_cliente


array([[False]])